<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 9. Decorators and Closures

16:06 - 17:06

A decorator is a callable that takes another function as an argument.

A decorator may perform some processing with the decorated function, and returns it or replaces it with another function or callable object.

In [ ]:
@decorate
def target():
  print('running target()')

is the same as:

In [ ]:
def target():
  print('running target()')

target = decorate(target)

In [2]:
def deco(func):
  def inner():
    print('running inner()')
  return inner # replace incoming func with inner function

In [3]:
@deco
def target():
  print('running target()')

target()

running inner()


In [4]:
target

<function __main__.deco.<locals>.inner()>

Strictly speaking, decorators are just syntactic sugar. Sometime that is actually convenient, especially when doing metaprogramming-changing program behavior at runtime.

* Decorators are executed immediately when a module is loaded

In [ ]:
# registry.py

registry = [] # registry will hold references to functions
              # decorated by @register

def register(func): # register takes a function as an argument
  print(f"running register({func})") # display what function is decorated
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

In [6]:
!python registry.py

running register(<function f1 at 0x7b5621466050>)
running register(<function f2 at 0x7b56214660e0>)
running main()
registry -> [<function f1 at 0x7b5621466050>, <function f2 at 0x7b56214660e0>]
running f1()
running f2()
running f3()


In [7]:
import registry

running register(<function f1 at 0x7afae6dd7e20>)
running register(<function f2 at 0x7afae6dd4a60>)


In [8]:
registry.registry

[<function registry.f1()>, <function registry.f2()>]

* The main point of the above example is to emphasize that function decorators are executed as soon as the module is imported, but the decorated functions only run when they are explicitly invoked.

## Registration Decorators

Two ununsual things of the above example:
 * The decorator function is defined in the same module as the decorated functions. A real decorator is usually defined in one module and applied to functions in other modules
 * The `register` decorator returns the same function passed as an argument. In practice, most decorators define an inner function and return it.

## Variable Scope Rules in Python

In [10]:
def f1(a):
  print(a) # local variable (defined as fcn param)
  print(b) # variable b that is not defined anywhere

f1(3)

3


NameError: name 'b' is not defined

In [11]:
b = 6

In [12]:
f1(3)

3
6


In [13]:
def f2(a):
  print(a)
  print(b)
  b = 9

In [14]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

Why error? When Python compiles the body of the function, it decides that `b` is a local variable because it is assigned within the function. The generated bytecode reflects this decision and will try to fetch `b` from local scope.

In [15]:
b = 6
def f3(a):
  global b
  print(a)
  print(b)
  b = 9

In [16]:
f3(3)

3
6


In [17]:
b

9

In [18]:
from dis import dis

In [19]:
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [20]:
dis(f2)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  4          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## Closures

Closures are not the same as anonymous functions. Many confuse them because of the parallel history of those features: defining functions inside functions is not so common or convenient, until you have anonymous functions. And closure only matter when you have nested functions.

Actually, a closure is a function (let's call if `f`) with an extended scope that encompasses variables referenced in the body of `f` that are not global variables or local variables of `f`. Such variables must come from the local scopoe of an outer function that encompasses `f`.

It does not matter whether the function is anonymous or not; what matters is that it can access nonglobal variables that are defined outside of its body.

In [23]:
# average_oo.py
class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total / len(self.series)

In [25]:
avg = Averager()
avg(10)

10.0

In [26]:
avg(11)

10.5

In [27]:
avg(12)

11.0

In [28]:
def make_averager():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total / len(series)

  return averager

Within `averager`, `series` is a free variable. This is a technical term meaning a variable that is not bound in the local scope.

In [29]:
avg = make_averager() # inner function of make_averager

In [30]:
avg(10)

10.0

In [31]:
avg(11)

10.5

In [32]:
avg(12)

11.0

In [33]:
avg.__code__ # attr that represents the compiled body of func

<code object averager at 0x7afadd244870, file "<ipython-input-28-5ed7c9c25689>", line 4>

In [34]:
avg.__code__.co_varnames

('new_value', 'total')

In [35]:
avg.__code__.co_freevars

('series',)

In [36]:
avg.__code__.co_freevars

('series',)

In [37]:
avg.__closure__

(<cell at 0x7afadd24b640: list object at 0x7afadd0e90c0>,)

In [40]:
avg.__closure__[0]

<cell at 0x7afadd24b640: list object at 0x7afadd0e90c0>

To summarize: a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

## The nonlocal Declaration

In [41]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    count += 1
    total += new_value
    return total / count

  return averager

In [42]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

With immutable types like numbers, strings, tuples, etc., all you can do is read, never update. If you try to rebind them, as in count = count + 1, then you are implicitly creating a local variable `count`. It is no longer a free variable, and therefore it is not saved in the closure.

To work around this, the `nonlocal` keyword was introduced in Python 3. If a new value is assigned to a `nonlocal` varaible, the binding stored in the closure is changed.

In [43]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count

  return averager

In [44]:
avg = make_averager()

In [45]:
avg(3)

3.0

In [46]:
avg(4)

3.5

In [47]:
avg(5)

4.0

### Variable Lookup Logic

When a function is defined, the Python bytecode compiler determines how to fetch a variable `x` that appears in it, based on three rules:
* If there is a `global x` declaration, `x` comes from and is assigned to the `x` global variable module.
* If there is a `nonlocal x` declaration, `x` comes from and is assigned to the `x` local variable of teh nearest surrounding function where x is defined.

## Implementing a Simple Decorator

In [49]:
# clockdeco0.py
import time

def clock(func):
  def clocked(*args): # define inner clocked to accept any # of pos args
    t0 = time.perf_counter()
    result = func(*args) # this line only works because the closure for
                         # clocked encompasses the func free variable
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
    return result
  return clocked


In [50]:
import time
from clockdeco0 import clock

@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n * factorial(n - 1)

if __name__ == '__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(0.123)
  print('*' * 40, 'Calling factorial(6)')
  print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.12326532s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000064s] factorial(1) -> 1
[0.00003146s] factorial(2) -> 2
[0.00005524s] factorial(3) -> 6
[0.00007822s] factorial(4) -> 24
[0.00010040s] factorial(5) -> 120
[0.00012609s] factorial(6) -> 720
6! =  720


### How it works

In [ ]:
@clock
def factorial(n):
  return 1 if n < 2 else n * factorial(n - 1)

# is actually equivalent to

def factorial(n):
  return 1 if n < 2 else n * factorial(n - 1)

factorial = clock(factorial)

17:24 - 17:40

In [51]:
import clockdeco_demo

In [52]:
clockdeco_demo.factorial.__name__

'clocked'

In [ ]:
# clockdeco.py
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.perf_counter()
    result = func(*args, **kwargs)
    elapsed = time.perf_counter() - t0
    name = func.__name__

    arg_lst = [repr(arg) for arg in args]
    arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
    arg_str = ', '.join(arg_lst)
    print(f"[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}")
    return result
  return clocked()


In [54]:
%load_ext autoreload
%autoreload 2


In [58]:
!python clockdeco_demo.py

**************************************** Calling snooze(.123)
args (0.123,)
kwargs {}
[0.12321354s] snooze(0.123) -> None
**************************************** Calling factorial(6)
args (6,)
kwargs {}
args (5,)
kwargs {}
args (4,)
kwargs {}
args (3,)
kwargs {}
args (2,)
kwargs {}
args (1,)
kwargs {}
[0.00000092s] factorial(1) -> 1
[0.00003536s] factorial(2) -> 2
[0.00005322s] factorial(3) -> 6
[0.00007781s] factorial(4) -> 24
[0.00010145s] factorial(5) -> 120
[0.00012157s] factorial(6) -> 720
6! =  720


In [59]:
import clockdeco_demo

In [60]:
clockdeco_demo.factorial.__name__

'clocked'